In [2]:
import logging
import datetime
from logging.handlers import TimedRotatingFileHandler
from logging.handlers import RotatingFileHandler
from sklearn.externals import joblib
from collections import OrderedDict
import sys
from dayu.hooks.oss_hook import OSSHook

from dayu.hooks.hive_server_hook import HiveServerHook
from dayu.hooks.hive_cli_hook import HiveCliHook

import pandas as pd 
import numpy as np
import time
import requests
import math
import logging
import datetime
import os

def split_table_name(datain):
    new_cols = []
    for column in datain.columns:
        if(len(column.split('.'))<2):
            return datain
        tb_name, col_name = column.split('.')
        new_cols.append((column, col_name))
    datain = datain.rename(columns=dict(new_cols))
    return datain

def read_from_hive2(output_file_name,insql,dtype):
    filename = output_file_name
    filepath = curr_dir+filename
    hive = HiveServerHook("warehouse_hive")
    hive.to_csv(insql,filepath , delimiter=',',lineterminator='\n', output_header=True)
    outdata = pd.read_csv(filepath, header=0,dtype=dtype)
    # 去除列名中带有的表名
#     outdata = split_table_name(outdata)
    return outdata

## 计算时间差
def date_time_sub(startTime,endTime,date_format):
    try:
        startTime= datetime.datetime.strptime(startTime,date_format)
        endTime= datetime.datetime.strptime(endTime,date_format)
        return (endTime - startTime).days
    except:
        pass

    # 计算车辆当前保值率
def computer_with_license_month(tar):
    try:
        license_month = tar['license_month']
        if(license_month<=12):
            #tar['keep_value'] = tar['year_1']
            return tar['year_1']
        else:
            year = license_month//12
            #当前年保值率
            keep_max = tar["year_"+str(int(year))]
            #下一年的保值率
            keep_min = tar["year_"+str(int(year+1))]

            #相比于上一年，已经过了几个月
            mon = license_month-12*year
            tem = (keep_max-keep_min)/12

            #tar['keep_value'] = round(keep_max - tem*mon, 4)
            return round(keep_max - tem*mon, 4)
    except:
        return tar["year_16"]

class Logger:       
    def __init__(self, logName, logFile):
        self._logger = logging.getLogger(logName)
        handler = logging.FileHandler(logFile)
        formatter = logging.Formatter('%(asctime)s ********* %(message)s')
        handler.setFormatter(formatter)
        self._logger.addHandler(handler)
        self._logger.setLevel(logging.INFO)

    def log(self, msg):
        if self._logger is not None:
            self._logger.info(msg)

def data3m_pinggu(data_df,col):
    total = data_df.shape[0]
    num_3 = data_df.loc[(data_df[col]>=-0.03 ) & (data_df[col]<=0.03 )].shape[0]
    print("P<3%: ",round(num_3/total,4))
    num_5 = data_df.loc[(data_df[col]>=-0.05 ) & (data_df[col]<=0.05 )].shape[0]
    print("P<5%: ",round(num_5/total,4))
    num_8 = data_df.loc[(data_df[col]>=-0.08 ) & (data_df[col]<=0.08 )].shape[0]
    print("P<8%: ",round(num_8/total,4))
    num_10 = data_df.loc[(data_df[col]>=-0.1 ) & (data_df[col]<=0.1 )].shape[0]
    print("P<10%: ",round(num_10/total,4))
    num_20 = data_df.loc[(data_df[col]>=-0.2 ) & (data_df[col]<=0.2 )].shape[0]
    print("P<20%: ",round(num_20/total,4))
    return round(num_3/total,4),round(num_5/total,4),round(num_8/total,4),round(num_10/total,4),round(num_20/total,4)
curr_date = str(datetime.datetime.now())[0:10]
logging.info('当前日期: %s'%(curr_date))
curr_dir = '/home/souche/qiongjiu/valuation/'
curr_dir = '/home/souche/qiongjiu/hgc/'
curr_date=str(datetime.datetime.now())[0:10]
hive_cli = HiveCliHook("warehouse_hive")

logger = Logger('model_service','./log/accurate_valuation_cyp_run_log.log')
logger.log("程序启动.............")


[2021-01-26 10:03:55,161] {<ipython-input-2-cbe05f263385>:101} INFO - 当前日期: 2021-01-26
[2021-01-26 10:03:55,188] {<ipython-input-2-cbe05f263385>:85} INFO - 程序启动.............


In [36]:
curr_date = '2021-01-25'
sql='''
select tt1.*,
       tt2.guide_price,
       tt3.brand_name_,
       tt3.series_name_,
       tt3.model_name_,
       tt4.environmental_standards 
from (SELECT t1.order_id,
             t1.order_price/1000000 order_price,
             t1.order_status,
             t1.auction_mode,
             t1.pay_time,
             t1.tenant_name,
             t1.seller_name,
             substr(t2.c_register_date,0,10) first_license_date,
             t2.c_rank_desc rank_desc,
             t2.c_rank_level rank_level,
             t2.c_color color_name,
             cast(t2.c_transfer_count as string) tradetimes,
             t2.c_mileage mileage,
             t2.c_car_model_id model_code,
             t2.c_car_model_name model_name,
             t2.c_series_name series_name,
             t2.c_series_id series_code,
             t2.c_brand brand_name,
             t2.c_brand_id brand_code,
             t2.license_simplify,
             t2.use_type purpose,
             t2.c_trade_code trade_code, 
             t5.is_version
      FROM(SELECT order_id,
                  c_goods_id,
                  total_price order_price,
                  seller_price,
                  order_status,
                  auction_mode,
                  goods_source,
                  create_time pay_time,
                  tenant_name,
                  seller_name,
                  source_type 
              FROM
                  edw_cyp.dwb_evt_tra_cypf_order_dd
              WHERE
                  ds = date_sub("'''+curr_date+'''",1) AND
                  c_goods_id IS NOT NULL 
                  and delete_tag = 0 
                  ) t1
      JOIN(SELECT c_id,
                  substr(CAST (c_register_date AS STRING),1,10)  c_register_date,
                  c_rank_desc,
                  c_rank_level,
                  c_color,
                  c_transfer_count,
                  c_mileage,
                  c_car_model_id,
                  c_car_model_name,
                  c_license,
                  c_register_code,
                  c_register_name,
                  c_register_province_code,
                  c_register_province_name,
                  c_min_offering_price,
                  c_operation_tag,
                  c_trade_code,
                  c_base_price,
                  c_buy_price,
                  c_auction_price,
                  c_series_name,
                  c_series_id ,
                  c_brand,
                  c_brand_id,
                  license_simplify,
                  use_type,
                  c_goods_source,
                  c_has_auction,
                  c_auction_num,
                  c_car_type,
                  c_offsite_car,
                  c_ca_status
              FROM 
                  edw_cyp.dwb_gds_cypd_goods_dd
              WHERE
                  ds =  date_sub("'''+curr_date+'''",1)) t2 
      ON t1.c_goods_id = t2.c_id 
      left join(select report_code,
                 product_id,
                 is_version,
                 product_type,
                 product_type_name
          from edw_cyp.dwb_evt_act_cypf_detect_order_dd
          where ds = date_sub("'''+curr_date+'''",1)) t5 
      on t2.c_trade_code = t5.report_code 
      where t2.c_register_date IS NOT NULL AND
            t2.c_mileage IS NOT NULL and 
            t2.c_car_model_id is not null and 
            t5.is_version = 0 and 
            t1.order_status = 80 and 
            t1.auction_mode in (2,4,1,1001,1011,1021,74) 
union 
SELECT car_id order_id,
    second_price/10000 order_price,
    '' order_status,
    '' auction_mode,
    publish_time pay_time,
    '' tenant_name,
    '' seller_name,
    substr(license_time,0,10) first_license_date,
    substr(RANK,1,1) rank_desc,
    CASE
        WHEN substr(RANK,2,2) IN ('30','35','40') THEN '35'
        WHEN substr(RANK,2,2) IN ('45','50','55') THEN '50'
        WHEN substr(RANK,2,2) IN ('60','65','70') THEN '65'
        WHEN substr(RANK,2,2) IN ('75','80','85') THEN '80'
        WHEN substr(RANK,2,2) IN ('90','95') THEN '95'
    END
    rank_level,
    color color_name,
    cast(transfer_times as string) tradetimes,
    mileage/10000 mileage,
    model_code,
    model_name,
    series_name,
    series_code,
    brand_name,
    brand_code,
    license_number license_simplify,
    use_property purpose,
    '' trade_code,
    '' is_version 
FROM
    dl_outrs.dl_vehicle_data_scavenger_ipbl_t_reptile_car_dd
WHERE
    ds = date_sub("'''+curr_date+'''",1) AND
    site ='yxpai' AND
    substr(publish_time,0,4) in ('2020','2019') and 
    model_code is not null ) tt1 
left join (select model_code,
                  guide_price 
           from db_data.ods_car_model_model_price 
           where ds =  date_sub("'''+curr_date+'''",1)) tt2 
on tt1.model_code = tt2.model_code 
left join (select brand_name brand_name_,
                  series_name series_name_,
                  model_code,
                  model_name model_name_ 
           from db_data.ods_car_model_model  
           where ds = date_sub("'''+curr_date+'''",1)) tt3  
on tt1.model_code = tt3.model_code 
left join (select model_code,
                  environmental_standards 
           from db_data.ods_car_model_model_parameter 
           where ds = date_sub("'''+curr_date+'''",1) ) tt4 
on tt1.model_code = tt4.model_code 

'''
dtype={'city_code':str}
mid_car_cyp_paimai_order = read_from_hive2('mid_car_cyp_paimai_order_dd',sql,dtype)

cyp_paimai_order_columns = []
for col in mid_car_cyp_paimai_order.columns:
    if len(col.split('.')) > 1:
        cyp_paimai_order_columns.append(col.split('.')[1])

    else:
        cyp_paimai_order_columns.append(col)
mid_car_cyp_paimai_order.columns = cyp_paimai_order_columns 
logger.log("读取数据完成.............")
# 557947



[2021-01-26 10:30:18,701] {hiveserver2:138} INFO - Using database default as default
[2021-01-26 10:30:18,837] {hive_server_hook:112} INFO - Running query: 
select tt1.*,
       tt2.guide_price,
       tt3.brand_name_,
       tt3.series_name_,
       tt3.model_name_,
       tt4.environmental_standards 
from (SELECT t1.order_id,
             t1.order_price/1000000 order_price,
             t1.order_status,
             t1.auction_mode,
             t1.pay_time,
             t1.tenant_name,
             t1.seller_name,
             substr(t2.c_register_date,0,10) first_license_date,
             t2.c_rank_desc rank_desc,
             t2.c_rank_level rank_level,
             t2.c_color color_name,
             cast(t2.c_transfer_count as string) tradetimes,
             t2.c_mileage mileage,
             t2.c_car_model_id model_code,
             t2.c_car_model_name model_name,
             t2.c_series_name series_name,
             t2.c_series_id series_code,
             t2.c_brand bran

[2021-01-26 10:31:47,274] {hive_server_hook:162} INFO - Written 340000 rows so far.
[2021-01-26 10:31:48,421] {hive_server_hook:162} INFO - Written 350000 rows so far.
[2021-01-26 10:31:49,680] {hive_server_hook:162} INFO - Written 360000 rows so far.
[2021-01-26 10:31:50,966] {hive_server_hook:162} INFO - Written 370000 rows so far.
[2021-01-26 10:31:52,217] {hive_server_hook:162} INFO - Written 380000 rows so far.
[2021-01-26 10:31:53,370] {hive_server_hook:162} INFO - Written 390000 rows so far.
[2021-01-26 10:31:54,599] {hive_server_hook:162} INFO - Written 400000 rows so far.
[2021-01-26 10:31:55,829] {hive_server_hook:162} INFO - Written 410000 rows so far.
[2021-01-26 10:31:57,143] {hive_server_hook:162} INFO - Written 420000 rows so far.
[2021-01-26 10:31:58,292] {hive_server_hook:162} INFO - Written 430000 rows so far.
[2021-01-26 10:31:59,546] {hive_server_hook:162} INFO - Written 440000 rows so far.
[2021-01-26 10:32:00,778] {hive_server_hook:162} INFO - Written 450000 rows 

/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [37]:
mid_car_cyp_paimai_order_back = mid_car_cyp_paimai_order.copy()
mid_car_cyp_paimai_order = mid_car_cyp_paimai_order_back.copy()


In [38]:
mid_car_cyp_paimai_order.loc[mid_car_cyp_paimai_order['model_code'] == '20415'].shape


(201, 28)

In [39]:
## 交易时间
mid_car_cyp_paimai_order = mid_car_cyp_paimai_order.loc[mid_car_cyp_paimai_order['pay_time'].notnull()]
mid_car_cyp_paimai_order['pay_time'] = mid_car_cyp_paimai_order['pay_time'].map(lambda x:str(x)[:10])
## 上牌时间
mid_car_cyp_paimai_order = mid_car_cyp_paimai_order.loc[mid_car_cyp_paimai_order['first_license_date'].notnull()]
mid_car_cyp_paimai_order['first_license_date'] = mid_car_cyp_paimai_order['first_license_date'].map(lambda x:str(x)[:10])
# 公里数
mid_car_cyp_paimai_order['mileage_std'] = mid_car_cyp_paimai_order['mileage'].astype('float')
mid_car_cyp_paimai_order['price'] = mid_car_cyp_paimai_order['order_price'].astype('float')

mid_car_cyp_paimai_order['license_time_month'] = mid_car_cyp_paimai_order['first_license_date'].map(lambda x:int(x.split('-')[1]))
mid_car_cyp_paimai_order['publish_time_month'] = mid_car_cyp_paimai_order['pay_time'].map(lambda x:int(x.split('-')[1]))
mid_car_cyp_paimai_order['license_year'] = mid_car_cyp_paimai_order['first_license_date'].map(lambda x:int(x.split('-')[0]))
mid_car_cyp_paimai_order['publish_time_year'] = mid_car_cyp_paimai_order['pay_time'].map(lambda x:int(x.split('-')[0]))

mid_car_cyp_paimai_order['license_month'] = mid_car_cyp_paimai_order['publish_time_year'].map(lambda x:int(x)) * 12 + \
                                 mid_car_cyp_paimai_order['publish_time_month']  - \
                                 mid_car_cyp_paimai_order['license_year'].map(lambda x:int(x)) * 12 - \
                                 mid_car_cyp_paimai_order['license_time_month'] + 1

mid_car_cyp_paimai_order['guide_price'] = mid_car_cyp_paimai_order['guide_price']/10000
## 删除model_name 为空的数据
mid_car_cyp_paimai_order = mid_car_cyp_paimai_order.loc[~mid_car_cyp_paimai_order['model_name_'].isnull()]
## 计算残值率
mid_car_cyp_paimai_order['residual'] = mid_car_cyp_paimai_order['price']/mid_car_cyp_paimai_order['guide_price']

mid_car_cyp_paimai_order.shape


(538197, 36)

In [42]:
car_model_date = '2020-12-31'
# 读取 001 处理好的数据
# model = pd.read_csv(curr_dir+car_model_date+"版车型参数及独热编码.csv", header=0, low_memory=False)
sql='''
select * from db_data.enterprise2_car_parameter_onehot 
where ds = date_sub("'''+car_model_date+'''",1)
'''
dtype={'city_code':str}
model = read_from_hive2('mid_car_cyp_paimai_order_dd',sql,dtype)

model_columns = []
for col in model.columns:
    if len(col.split('.')) > 1:
        model_columns.append(col.split('.')[1])

    else:
        model_columns.append(col)
model.columns = model_columns 

# 读取可估价的车型
keep_value = model[['model_code', 'model_year', 'new_car_price','year_1', 'year_2', 'year_3', 'year_4', 'year_5', 'year_6',
                     'year_7', 'year_8', 'year_9', 'year_10', 'year_11', 'year_12', 'year_13', 
                     'year_14', 'year_15', 'year_16','rate','rate_count']]
print("合并前数据数目为:%d, 车型数量为：%d" % (mid_car_cyp_paimai_order.shape[0], len(mid_car_cyp_paimai_order['model_code'].unique())))
mid_car_cyp_paimai_order['model_code'] = mid_car_cyp_paimai_order['model_code'].astype('str')
mid_car_cyp_paimai_order = pd.merge(mid_car_cyp_paimai_order, keep_value, on='model_code', how='inner')
print("<---------------------------------------->")
print("合并后数据数目为:%d, 车型数量为：%d" % (mid_car_cyp_paimai_order.shape[0], len(mid_car_cyp_paimai_order['model_code'].unique())))


[2021-01-26 10:32:41,536] {hiveserver2:138} INFO - Using database default as default
[2021-01-26 10:32:41,673] {hive_server_hook:112} INFO - Running query: 
select * from db_data.enterprise2_car_parameter_onehot 
where ds = date_sub("2020-12-31",1)

[2021-01-26 10:32:48,640] {hive_server_hook:162} INFO - Written 10000 rows so far.
[2021-01-26 10:32:55,404] {hive_server_hook:162} INFO - Written 20000 rows so far.
[2021-01-26 10:33:02,145] {hive_server_hook:162} INFO - Written 30000 rows so far.
[2021-01-26 10:33:08,954] {hive_server_hook:162} INFO - Written 40000 rows so far.
[2021-01-26 10:33:15,282] {hive_server_hook:162} INFO - Written 50000 rows so far.
[2021-01-26 10:33:22,053] {hive_server_hook:162} INFO - Written 60000 rows so far.
[2021-01-26 10:33:25,801] {hive_server_hook:162} INFO - Written 65935 rows so far.
[2021-01-26 10:33:25,871] {hiveserver2:265} INFO - Closing active operation
[2021-01-26 10:33:25,887] {hive_server_hook:163} INFO - Done. Loaded a total of 65935 rows.


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


合并前数据数目为:538197, 车型数量为：22621
<---------------------------------------->
合并后数据数目为:504993, 车型数量为：16999


In [44]:
##======================================== ##
##       数据清洗,剔除峰值数据
##======================================== ##
mid_car_cyp_paimai_order['pay_time_year'] = mid_car_cyp_paimai_order['pay_time'].map(lambda x:x[:4])

dl_site_ts_order_clean_count = mid_car_cyp_paimai_order[['model_code','license_year','pay_time_year','price','rank_desc','rank_level']].\
groupby(['model_code','license_year','pay_time_year','rank_desc','rank_level']).count().reset_index().rename(columns={'price':'count'})

dl_site_ts_order_clean_mean = mid_car_cyp_paimai_order[['model_code','license_year','pay_time_year','price','rank_desc','rank_level']].\
groupby(['model_code','license_year','pay_time_year','rank_desc','rank_level']).mean().reset_index().rename(columns={'price':'mean'})

dl_site_ts_order_clean_std = mid_car_cyp_paimai_order[['model_code','license_year','pay_time_year','price','rank_desc','rank_level']].\
groupby(['model_code','license_year','pay_time_year','rank_desc','rank_level']).std().reset_index().rename(columns={'price':'std'})

dl_site_ts_order_clean_median = mid_car_cyp_paimai_order[['model_code','license_year','pay_time_year','price','rank_desc','rank_level']].\
groupby(['model_code','license_year','pay_time_year','rank_desc','rank_level']).median().reset_index().rename(columns={'price':'median'})

dl_site_ts_order_clean_min = mid_car_cyp_paimai_order[['model_code','license_year','pay_time_year','price','rank_desc','rank_level']].\
groupby(['model_code','license_year','pay_time_year','rank_desc','rank_level']).min().reset_index().rename(columns={'price':'min_price'})

dl_site_ts_order_clean_max = mid_car_cyp_paimai_order[['model_code','license_year','pay_time_year','price','rank_desc','rank_level']].\
groupby(['model_code','license_year','pay_time_year','rank_desc','rank_level']).max().reset_index().rename(columns={'price':'max_price'})

dl_site_ts_order_clean = pd.merge(mid_car_cyp_paimai_order,dl_site_ts_order_clean_count,on=['model_code','license_year','pay_time_year','rank_desc','rank_level'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_mean,on=['model_code','license_year','pay_time_year','rank_desc','rank_level'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_std,on=['model_code','license_year','pay_time_year','rank_desc','rank_level'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_min,on=['model_code','license_year','pay_time_year','rank_desc','rank_level'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_max,on=['model_code','license_year','pay_time_year','rank_desc','rank_level'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_median,on=['model_code','license_year','pay_time_year','rank_desc','rank_level'],how='left')

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['mean'] * 0.3 < dl_site_ts_order_clean['std']),'std'] = dl_site_ts_order_clean['mean'] * 0.3

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median'] > dl_site_ts_order_clean['mean']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['price'] < (dl_site_ts_order_clean['mean'] - dl_site_ts_order_clean['std'])),'delete_flag'] = 1

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median'] < dl_site_ts_order_clean['mean']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['price'] > (dl_site_ts_order_clean['mean'] + dl_site_ts_order_clean['std'])),'delete_flag'] = 1

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median'] > dl_site_ts_order_clean['mean']) & 
#                            (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['price'] < (dl_site_ts_order_clean['mean'] - dl_site_ts_order_clean['std'])),
                           'price'] = dl_site_ts_order_clean['mean'] - dl_site_ts_order_clean['std']

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median'] < dl_site_ts_order_clean['mean']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['price'] > (dl_site_ts_order_clean['mean'] + dl_site_ts_order_clean['std'])),
                           'price'] = dl_site_ts_order_clean['mean'] + dl_site_ts_order_clean['std']
## 剔除数据
dl_site_ts_order_clean = dl_site_ts_order_clean.loc[~(dl_site_ts_order_clean['delete_flag'] == 1)]
dl_site_ts_order_clean.shape


(446635, 64)

In [46]:
##======================================== ##
##       针对数据量非常少的数据清洗
##======================================== ##

dl_site_ts_order_clean['pay_time_year'] = dl_site_ts_order_clean['pay_time'].map(lambda x:x[:4])

dl_site_ts_order_clean_count = dl_site_ts_order_clean[['model_code','license_year','pay_time_year','price']].\
groupby(['model_code','license_year','pay_time_year']).count().reset_index().rename(columns={'price':'count_1'})

dl_site_ts_order_clean_mean = dl_site_ts_order_clean[['model_code','license_year','pay_time_year','price']].\
groupby(['model_code','license_year','pay_time_year']).mean().reset_index().rename(columns={'price':'mean_1'})

dl_site_ts_order_clean_std = dl_site_ts_order_clean[['model_code','license_year','pay_time_year','price']].\
groupby(['model_code','license_year','pay_time_year']).std().reset_index().rename(columns={'price':'std_1'})

dl_site_ts_order_clean_median = dl_site_ts_order_clean[['model_code','license_year','pay_time_year','price']].\
groupby(['model_code','license_year','pay_time_year']).median().reset_index().rename(columns={'price':'median_1'})

dl_site_ts_order_clean_min = dl_site_ts_order_clean[['model_code','license_year','pay_time_year','price']].\
groupby(['model_code','license_year','pay_time_year']).min().reset_index().rename(columns={'price':'min_price_1'})

dl_site_ts_order_clean_max = dl_site_ts_order_clean[['model_code','license_year','pay_time_year','price']].\
groupby(['model_code','license_year','pay_time_year']).max().reset_index().rename(columns={'price':'max_price_1'})

dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_count,on=['model_code','license_year','pay_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_mean,on=['model_code','license_year','pay_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_std,on=['model_code','license_year','pay_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_min,on=['model_code','license_year','pay_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_max,on=['model_code','license_year','pay_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_median,on=['model_code','license_year','pay_time_year'],how='left')

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['mean_1'] * 0.3 < dl_site_ts_order_clean['std_1']),'std_1'] = dl_site_ts_order_clean['mean_1'] * 0.3

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median_1'] > dl_site_ts_order_clean['mean_1']) & 
#                           (dl_site_ts_order_clean['mean_1']*0.5 <= dl_site_ts_order_clean['std_1']) & 
                          (dl_site_ts_order_clean['price'] < (dl_site_ts_order_clean['mean_1'] - 1.5*dl_site_ts_order_clean['std_1'])),'delete_flag_1'] = 1
dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median_1'] < dl_site_ts_order_clean['mean_1']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['price'] > (dl_site_ts_order_clean['mean_1'] + 1.5*dl_site_ts_order_clean['std_1'])),'delete_flag_1'] = 1

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median_1'] > dl_site_ts_order_clean['mean_1']) & 
#                            (dl_site_ts_order_clean['mean_1']*0.5 <= dl_site_ts_order_clean['std_1']) & 
                          (dl_site_ts_order_clean['price'] < (dl_site_ts_order_clean['mean_1'] - 1.5*dl_site_ts_order_clean['std_1'])),
                           'price'] = dl_site_ts_order_clean['mean_1'] - 1.3*dl_site_ts_order_clean['std_1']

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median_1'] < dl_site_ts_order_clean['mean_1']) & 
#                           (dl_site_ts_order_clean['mean_1']*0.5 <= dl_site_ts_order_clean['std_1']) & 
                          (dl_site_ts_order_clean['price'] > (dl_site_ts_order_clean['mean_1'] + 1.5*dl_site_ts_order_clean['std_1'])),
                           'price'] = dl_site_ts_order_clean['mean_1'] + 1.5*dl_site_ts_order_clean['std_1']

## 剔除数据
# dl_site_ts_order_clean = dl_site_ts_order_clean.loc[~(dl_site_ts_order_clean['delete_flag_1'] == 1)]
dl_site_ts_order_clean.shape



(446635, 71)

In [47]:
##======================================== ##
##       数据清洗,剔除异常数据
##======================================== ##
for col in ['median_1','mean_1','std_1','count_1','min_price_1','max_price_1']:
    if col in dl_site_ts_order_clean.columns:
        del dl_site_ts_order_clean[col]
dl_site_ts_order_clean['pay_time_year'] = dl_site_ts_order_clean['pay_time'].map(lambda x:x[:4])

dl_site_ts_order_clean_count = dl_site_ts_order_clean[['model_code','license_year','pay_time_year','price']].\
groupby(['model_code','license_year','pay_time_year']).count().reset_index().rename(columns={'price':'count_1'})

dl_site_ts_order_clean_mean = dl_site_ts_order_clean[['model_code','license_year','pay_time_year','price']].\
groupby(['model_code','license_year','pay_time_year']).mean().reset_index().rename(columns={'price':'mean_1'})

dl_site_ts_order_clean_std = dl_site_ts_order_clean[['model_code','license_year','pay_time_year','price']].\
groupby(['model_code','license_year','pay_time_year']).std().reset_index().rename(columns={'price':'std_1'})

dl_site_ts_order_clean_median = dl_site_ts_order_clean[['model_code','license_year','pay_time_year','price']].\
groupby(['model_code','license_year','pay_time_year']).median().reset_index().rename(columns={'price':'median_1'})

dl_site_ts_order_clean_min = dl_site_ts_order_clean[['model_code','license_year','pay_time_year','price']].\
groupby(['model_code','license_year','pay_time_year']).min().reset_index().rename(columns={'price':'min_price_1'})

dl_site_ts_order_clean_max = dl_site_ts_order_clean[['model_code','license_year','pay_time_year','price']].\
groupby(['model_code','license_year','pay_time_year']).max().reset_index().rename(columns={'price':'max_price_1'})

dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_count,on=['model_code','license_year','pay_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_mean,on=['model_code','license_year','pay_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_std,on=['model_code','license_year','pay_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_min,on=['model_code','license_year','pay_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_max,on=['model_code','license_year','pay_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_median,on=['model_code','license_year','pay_time_year'],how='left')

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['mean_1'] * 0.3 < dl_site_ts_order_clean['std_1']),'std_1'] = dl_site_ts_order_clean['mean_1'] * 0.3

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['price'] < (dl_site_ts_order_clean['mean_1'] - 1.5*dl_site_ts_order_clean['std_1'])),'delete_flag_2'] = 1

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['price'] > (dl_site_ts_order_clean['mean_1'] + 1.5*dl_site_ts_order_clean['std_1'])),'delete_flag_2'] = 1

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['price'] < (dl_site_ts_order_clean['mean_1'] - 1.5*dl_site_ts_order_clean['std_1'])),
                           'price'] = dl_site_ts_order_clean['mean_1'] - 1.5*dl_site_ts_order_clean['std_1']

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['price'] > (dl_site_ts_order_clean['mean_1'] + 1.5*dl_site_ts_order_clean['std_1'])),
                           'price'] = dl_site_ts_order_clean['mean_1'] + 1.5*dl_site_ts_order_clean['std_1']

dl_site_ts_order_clean.shape

(446635, 72)

In [48]:
mid_car_cyp_paimai_order = dl_site_ts_order_clean.copy()
## 计算残值率
mid_car_cyp_paimai_order['residual'] = mid_car_cyp_paimai_order['price']/mid_car_cyp_paimai_order['guide_price']
## 删除不合理的数据和不估值的数据
data_m = mid_car_cyp_paimai_order[(mid_car_cyp_paimai_order['mileage_std']>=0.01) & (mid_car_cyp_paimai_order['mileage_std']<=60)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))
data_m = data_m[(data_m['price']>=0.1) & (data_m['price']<=300)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))
data_m = data_m[(data_m['license_month']>=1) & (data_m['license_month']<=180)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))
# data_m = data_m[(data_m['residual']>=0.05) & (data_m['residual']<1)]
# data_m = data_m[(data_m['model_year']>=2005)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))
data_m = data_m[data_m['license_year']>=2005]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))
data_m = data_m[(data_m['new_car_price']>=1) & (data_m['new_car_price']<=500)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))


增加限制后，数据数量为:445544, 车型数量为:16945
增加限制后，数据数量为:444834, 车型数量为:16915
增加限制后，数据数量为:438924, 车型数量为:16764
增加限制后，数据数量为:438924, 车型数量为:16764
增加限制后，数据数量为:432519, 车型数量为:16591
增加限制后，数据数量为:432494, 车型数量为:16588


In [49]:
# 计算衍生变量
data_m['per_mile'] = round(data_m['mileage_std']/(data_m['license_month']/12),2)
data_m['year_err'] = data_m['license_year'] - data_m['model_year']
data_m = data_m[(data_m['year_err']>=-2) & (data_m['year_err']<=6)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))


增加限制后，数据数量为:430765, 车型数量为:16504


In [50]:
##======================================== ##
##       车牌的前两位转化为省份，并编码
##======================================== ##
data_m = data_m.loc[~data_m['license_simplify'].isnull()]
data_m['license_simplify'] = data_m['license_simplify'].map(lambda x:str(x)[:2])
# 读取编码完成的省份数据
province = pd.read_csv("../Valuation/province_new.csv", header=0, low_memory=False) 
licence = pd.read_excel("licence.xlsx")
licence = licence.rename(columns={"prov":"province_name","city":"city_name"})
licence.head()
if 'province_name' in data_m.columns:
    del data_m['province_name']
if 'city_name' in data_m.columns:
    del data_m['city_name']
    
data_m = pd.merge(data_m,licence[['license','province_name','city_name']],left_on='license_simplify',right_on='license',how='left')
data2m= pd.merge(data_m, province, how='left', on='province_name')


In [51]:
##======================================== ##
##       计算保值率
##======================================== ##
data2m['keep_value'] = data2m[['license_month','year_1', 'year_2', 'year_3', 'year_4','year_5', 'year_6', 'year_7', 'year_8', 'year_9', 'year_10', 'year_11','year_12', 'year_13', 'year_14', 'year_15', 'year_16']].to_dict(orient='records')
data2m['keep_value'] = data2m['keep_value'].map(lambda tar:computer_with_license_month(tar))

data3m = data2m.drop(['year_1', 'year_2', 'year_3', 'year_4', 
                    'year_5', 'year_6', 'year_7', 'year_8', 
                    'year_9', 'year_10', 'year_11', 'year_12', 
                    'year_13', 'year_14', 'year_15', 'year_16'], axis=1)
# quality = pd.read_csv(curr_dir+car_model_date+"版车型参数及独热编码.csv", header=0, low_memory=False)
quality = model[['model_code', 'quality_mile', 'quality_year']]
quality.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65935 entries, 0 to 65934
Data columns (total 3 columns):
model_code      65935 non-null object
quality_mile    65935 non-null float64
quality_year    65935 non-null float64
dtypes: float64(2), object(1)
memory usage: 1.5+ MB


In [52]:
##======================================== ##
##       计算是过质保
##======================================== ##
data3m = pd.merge(data3m, quality, how='left', on='model_code')
def get_quality(license_month,mileage_std,quality_mile,quality_year):
    year = round(license_month/12, 2)
    if mileage_std < quality_mile and year< quality_year:
        return 1
    else:
        return 0
       
data3m['quality'] = list(map(lambda license_month,mileage_std,quality_mile,quality_year:get_quality(license_month,mileage_std,quality_mile,quality_year),
          data3m['license_month'],data3m['mileage_std'],data3m['quality_mile'],data3m['quality_year']))
data3m = data3m.drop(['quality_mile', 'quality_year'], axis=1)


In [53]:
## 对数转化
# math.log 效率太低了，换成 np.log 之前是直接在取数的时候换的，以下code 需要测试确认
data3m['mile_log']=round(np.log2(data3m['mileage_std']), 4)
data3m['license_month_log']=round(np.log2(data3m['license_month']), 4)
# data3m['license_month_log'] = data3m['license_month'].map(lambda x:round(math.log(x,100),2))
data3m['new_car_price_log']=round(np.log2(data3m['new_car_price']), 4)


In [54]:
##======================================== ##
##       增加车型参数编码
##======================================== ##
# car = pd.read_csv(curr_dir+car_model_date+"版车型参数及独热编码.csv", header = 0, low_memory=False)
car=model.drop(['model_year','new_car_price', 'driving_mode','gear_box_type','country_id',
 'import_type','intake_type', 'fuel_form','quality_mile', 'quality_year',
 'year_1','year_2','year_3','year_4','year_5','year_6','year_7','year_8',
 'year_9', 'year_10','year_11','year_12','year_13','year_14','year_15',
 'year_16', 'car_body','rate','rate_count'],axis=1)
data3m = pd.merge(data3m, car, how="inner", on="model_code")


In [55]:
##======================================== ##
##       增加车型名称编码
##======================================== ##
# vec = pd.read_csv(curr_dir+car_model_date+"版-TF-IDF权重的词向量.csv", dtype={'model_code': str}, header = 0)
sql='''
select * from db_data.enterprise2_model_name_word2vec 
where ds = date_sub("'''+car_model_date+'''",1)
'''
dtype={'city_code':str}
vec = read_from_hive2('vec',sql,dtype)

vec_columns = []
for col in vec.columns:
    if len(col.split('.')) > 1:
        vec_columns.append(col.split('.')[1])

    else:
        vec_columns.append(col)
vec.columns = vec_columns 

vec = vec.drop(['model_name'], axis=1)
data3m = pd.merge(data3m, vec, how="inner", on="model_code")


[2021-01-26 10:34:12,638] {hiveserver2:138} INFO - Using database default as default
[2021-01-26 10:34:12,780] {hive_server_hook:112} INFO - Running query: 
select * from db_data.enterprise2_model_name_word2vec 
where ds = date_sub("2020-12-31",1)

[2021-01-26 10:34:17,800] {hive_server_hook:162} INFO - Written 10000 rows so far.
[2021-01-26 10:34:22,413] {hive_server_hook:162} INFO - Written 20000 rows so far.
[2021-01-26 10:34:27,101] {hive_server_hook:162} INFO - Written 30000 rows so far.
[2021-01-26 10:34:31,782] {hive_server_hook:162} INFO - Written 40000 rows so far.
[2021-01-26 10:34:36,128] {hive_server_hook:162} INFO - Written 50000 rows so far.
[2021-01-26 10:34:40,741] {hive_server_hook:162} INFO - Written 60000 rows so far.
[2021-01-26 10:34:43,388] {hive_server_hook:162} INFO - Written 65935 rows so far.
[2021-01-26 10:34:43,746] {hiveserver2:265} INFO - Closing active operation
[2021-01-26 10:34:43,764] {hive_server_hook:163} INFO - Done. Loaded a total of 65935 rows.


In [56]:
##======================================== ##
##       交易距今时间
##======================================== ##
data3m['pay_time'] = data3m['pay_time'].map(lambda x:x[:10])
data3m['days'] = list(map(lambda x:date_time_sub(x,'2021-12-31',"%Y-%m-%d"),data3m['pay_time']))
import math
data3m['days_log'] = data3m['days'].map(lambda x:round(math.log(x,100),1))
data3m['month'] = data3m['days'].map(lambda x:round(x/30))



In [58]:
fearture_columns = ['month', 'license_year', 'model_year', 'rate', 'rate_count', 'per_mile', 'year_err',
                    'province_name0', 'province_name1', 'province_name2', 'province_name3', 'province_name4',
                    'province_name5', 'province_name6', 'province_name7', 'province_name8', 'province_name9', 
                    'province_name10', 'province_name11', 'province_name12', 'province_name13', 'province_name14',
                    'province_name15', 'province_name16', 'province_name17', 'province_name18', 'province_name19', 
                    'province_name20', 'province_name21', 'province_name22', 'province_name23', 'province_name24', 
                    'province_name25', 'province_name26', 'province_name27', 'province_name28', 'province_name29', 
                    'province_name30', 'keep_value', 'quality', 'mile_log', 'license_month_log', 'new_car_price_log',
                    'wheel_base', 'length', 'height', 'width', 'max_torque', 'max_power', 'engine_volume_l', 'cylinder_number', 
                    'seat_number_top', 'driving_mode0', 'driving_mode1', 'driving_mode2', 'driving_mode3', 'driving_mode4', 
                    'driving_mode5', 'driving_mode6', 'driving_mode7', 'driving_mode8', 'driving_mode9', 'gear_box_type0', 
                    'gear_box_type1', 'gear_box_type2', 'gear_box_type3', 'gear_box_type4', 'gear_box_type5', 'gear_box_type6', 
                    'gear_box_type7', 'gear_box_type8', 'gear_box_type9', 'gear_box_type10', 'country_id0', 'country_id1', 
                    'country_id2', 'country_id3', 'country_id4', 'country_id5', 'country_id6', 'country_id7', 'country_id8', 
                    'country_id9', 'country_id10', 'country_id11', 'country_id12', 'country_id13', 'country_id14', 'import_type0', 
                    'import_type1', 'intake_type0', 'intake_type1', 'intake_type2', 'intake_type3', 'intake_type4', 'intake_type5', 
                    'intake_type6', 'intake_type7', 'fuel_form0', 'fuel_form1', 'fuel_form2', 'fuel_form3', 'fuel_form4', 
                    'fuel_form5', 'fuel_form6', 'fuel_form7', 'fuel_form8', 'car_body0', 'car_body1', 'car_body2', 'car_body3',
                    'car_body4', 'car_body5', 'car_body6', 'car_body7', 'car_body8', 'car_body9', 'car_body10', 'car_body11', 
                    'series_level0', 'series_level1', 'series_level2', 'series_level3', 'series_level4', 'series_level5', 
                    'series_level6', 'series_level7', 'series_level8', 'series_level9', 'series_level10', 'series_level11', 
                    'series_level12', 'series_level13', 'series_level14', 'series_level15', 'series_level16', 'series_level17', 
                    'series_level18', 'series_level19', 'series_level20', 'series_level21', 'series_level22', 'series_level23', 
                    'series_level24', 'series_level25', 'series_level26', 'series_level27', 'series_level28', 'series_level29',
                    'series_level30', 'series_level31', 'series_level32', 'series_level33', 'series_level34', 'series_level35',
                    'series_level36', 'series_level37', 'series_level38', 'series_level39', 'series_level40', 'series_level41', 
                    'series_level42', 'series_level43', 'series_level44', 'series_level45', 'series_level46', 'series_level47', 
                    'series_level48', 'series_level49', 'series_level50', 'series_level51', 'series_level52', 'series_level53', 
                    'series_level54', 'series_level55', 'series_level56', 'series_level57', 'series_level58', 'series_level59', 
                    'series_level60', 'series_level61', 'wordvec0', 'wordvec1', 'wordvec2', 'wordvec3', 'wordvec4', 'wordvec5', 
                    'wordvec6', 'wordvec7', 'wordvec8', 'wordvec9', 'wordvec10', 'wordvec11', 'wordvec12', 'wordvec13', 'wordvec14',
                    'wordvec15', 'wordvec16', 'wordvec17', 'wordvec18', 'wordvec19', 'wordvec20', 'wordvec21', 'wordvec22', 'wordvec23',
                    'wordvec24', 'wordvec25', 'wordvec26', 'wordvec27', 'wordvec28', 'wordvec29', 'wordvec30', 'wordvec31', 'wordvec32', 
                    'wordvec33', 'wordvec34', 'wordvec35', 'wordvec36', 'wordvec37', 'wordvec38', 'wordvec39', 'wordvec40', 'wordvec41',
                    'wordvec42', 'wordvec43', 'wordvec44', 'wordvec45', 'wordvec46', 'wordvec47', 'wordvec48', 'wordvec49', 'wordvec50',
                    'wordvec51', 'wordvec52', 'wordvec53', 'wordvec54', 'wordvec55', 'wordvec56', 'wordvec57', 'wordvec58', 'wordvec59', 
                    'wordvec60', 'wordvec61', 'wordvec62', 'wordvec63', 'wordvec64', 'wordvec65', 'wordvec66', 'wordvec67', 'wordvec68', 
                    'wordvec69', 'wordvec70', 'wordvec71', 'wordvec72', 'wordvec73', 'wordvec74', 'wordvec75', 'wordvec76', 'wordvec77',
                    'wordvec78', 'wordvec79', 'wordvec80', 'wordvec81', 'wordvec82', 'wordvec83', 'wordvec84', 'wordvec85', 'wordvec86',
                    'wordvec87', 'wordvec88', 'wordvec89', 'wordvec90', 'wordvec91', 'wordvec92', 'wordvec93', 'wordvec94', 'wordvec95', 
                    'wordvec96', 'wordvec97', 'wordvec98', 'wordvec99']

label = 'residual'


In [59]:
##======================================== ##
##       对优信拍数据处理
##======================================== ##
data3m['purpose'] = data3m['purpose'].map(lambda x:str(x))
data3m.loc[data3m['purpose'] == '非营运','purpose'] = '1'
data3m.loc[data3m['purpose'] == '租赁公司非营运','purpose'] = '5'
data3m.loc[data3m['purpose'] == '营转非','purpose'] = '2'
data3m.loc[data3m['purpose'] == '租赁','purpose'] = '5' 
data3m.loc[data3m['purpose'] == '营运','purpose'] = '0'


In [60]:
##======================================== ##
##       对精准估值的过户、营运、颜色、车况处理
##======================================== ##
## rank_desc,rank_level,color_name,tradetimes,'purpose','isblister'
## purpose 车辆使用性质（10非营运、20 营运  30  营转非  40  租赁  50  教练...
## isblister 是否水泡车辆  0 否  1 是  默认 0
## isaccident 是否事故车辆 0:非事故车 1:事故车
# data3m.loc[data3m['purpose'] == 10,'purpose'] = '10'
# data3m = data3m.loc[data3m['purpose'].isin(['10', '20', '30', '40', '50'])]
## 产品模式，竞拍玩法 场次产品线：1-现场拍、2-快拍、4-易拍、5-线上撮合、72-面对面交易、77-暗拍、
##1001-车道拍、1011-超级拍（现场拍）、1021-超级拍(车道拍)、 25-现场拍、35-线下交易、36-公司收购、7-原公司收购、
##73-卖家直拍、74-随时拍、76-自行交易、1002-批售暗拍

data3m.loc[data3m['purpose'] == '1','purpose'] = '10'
data3m.loc[data3m['purpose'] == '2','purpose'] = '30'  
data3m.loc[data3m['purpose'] == '0','purpose'] = '20'
data3m.loc[data3m['purpose'] == '5','purpose'] = '40'
data3m.loc[data3m['purpose'] == '6','purpose'] = '50'
# data3m.loc[data3m['purpose'] == 10,'purpose'] = '10'
data3m = data3m.loc[data3m['purpose'].isin(['10', '20', '30', '40', '50'])]
print("使用性质",data3m.shape)
data3m = data3m.loc[data3m['rank_desc'].isin(['A','B','C','D'])]

##处理过户次数
data3m['tradetimes'] = data3m['tradetimes'].fillna(-1)
tradetimes_df = data3m[['tradetimes','model_code']].groupby('tradetimes').count().reset_index().rename(columns={'model_code':'tradetimes_count'})
tradetimes_list = list(tradetimes_df.loc[tradetimes_df['tradetimes_count']>2]['tradetimes'])
data3m = data3m.loc[data3m['tradetimes'].isin(tradetimes_list)]
data3m['tradetimes'] = data3m['tradetimes'].map(lambda x:int(x))
data3m.loc[data3m['tradetimes']>5,'tradetimes'] = 5
data3m['tradetimes'] = data3m['tradetimes'].map(lambda x:str(int(x)))

data3m['rank_level'] = data3m['rank_level'].map(lambda x:str(int(x)))

data3m['color_name'] = data3m['color_name'].map(lambda x:str(x).replace('色','').replace('/','').replace('／',''))
data3m.loc[data3m['color_name'].isin(['nan', '不详','其他颜']),'color_name'] = '其他'
data3m.loc[~data3m['color_name'].isin(['银','粉','绿','紫','棕','黄','蓝','红','白','黑','灰','其他']),
           'color_name'] = '多色'


使用性质 (430341, 352)


In [62]:
##======================================== ##
##       增加排放标准和排放标准数据处理
##======================================== ##
data3m.loc[data3m['environmental_standards'].isnull(),'environmental_standards'] = '14'
environmental_standards_dict = {"0":"国II","1":"国IV","2":"国III","3":"国IV","4":"国III","5":"国III","6":"国II",
                            "7":"国IV","8":"国IV","9":"国V","10":"国IV","11":"国III","12":"国V","13":"国V",
                            "14":"未知","15":"国VI","16":"国IV","17":"国V","19":"国V","20":"国V","21":"国V","22":"国I",
                            "23":"国IV","24":"国IV","25":"国IV","26":"国V","27":"国III","28":"国V","30":"国VI",
                            "31":"国VI","32":"国VI","33":"未知","34":"化油器","35":"纯电动","36":"国V","37":"国VI",
                            "38":"国IV","39":"国III","40":"国V","41":"国III","42":"国IV"}

environmental_standards_odb = ["5","7","8","11","12","16","21","23","24","26","27","32","41"]
data3m['environmental'] = data3m['environmental_standards'].map(lambda x:environmental_standards_dict[str(int(x))])

data3m.loc[data3m['environmental_standards'].map(lambda x:str(int(x))).isin(environmental_standards_odb),'OBD'] = 1 
data3m['OBD'] = data3m['OBD'].fillna(0)


In [63]:
## 对精准估值的参数进行编码
columns_dummies = pd.get_dummies(data3m[['rank_desc','rank_level','color_name','tradetimes','purpose']]) # ,'environmental'
data3m = pd.concat([data3m,columns_dummies],axis=1)


In [64]:
data3m['per_mile_log'] = data3m['per_mile'].map(lambda x:math.log(round(x,1)+1,2))
data3m['keep_value_log'] = data3m['keep_value'].map(lambda x:math.log(x))


In [66]:
## 定义特征
fearture_columns = ['month', 'license_year', 'model_year','license_month_log', 'rate', 'rate_count', 'per_mile_log', 'year_err',
                    'province_name0', 'province_name1', 'province_name2', 'province_name3', 'province_name4',
                    'province_name5', 'province_name6', 'province_name7', 'province_name8', 'province_name9', 
                    'province_name10', 'province_name11', 'province_name12', 'province_name13', 'province_name14',
                    'province_name15', 'province_name16', 'province_name17', 'province_name18', 'province_name19', 
                    'province_name20', 'province_name21', 'province_name22', 'province_name23', 'province_name24', 
                    'province_name25', 'province_name26', 'province_name27', 'province_name28', 'province_name29', 
                    'province_name30', 'keep_value_log', 'quality', 'new_car_price_log', 
                    'wheel_base', 'length', 'height', 'width', 'max_torque', 'max_power', 'cylinder_number', 
                    'seat_number_top', 'driving_mode0', 'driving_mode1', 'driving_mode2', 'driving_mode3', 'driving_mode4', 
                    'driving_mode5', 'driving_mode6', 'driving_mode7', 'driving_mode8', 'driving_mode9', 'gear_box_type0', 
                    'gear_box_type1', 'gear_box_type2', 'gear_box_type3', 'gear_box_type4', 'gear_box_type5', 'gear_box_type6', 
                    'gear_box_type7', 'gear_box_type8', 'gear_box_type9', 'gear_box_type10', 'country_id0', 'country_id1', 
                    'country_id2', 'country_id3', 'country_id4', 'country_id5', 'country_id6', 'country_id7', 'country_id8', 
                    'country_id9', 'country_id10', 'country_id11', 'country_id12', 'country_id13', 'country_id14', 'import_type0', 
                    'import_type1', 'intake_type0', 'intake_type1', 'intake_type2', 'intake_type3', 'intake_type4', 'intake_type5', 
                    'intake_type6', 'intake_type7', 'fuel_form0', 'fuel_form1', 'fuel_form2', 'fuel_form3', 'fuel_form4', 
                    'fuel_form5', 'fuel_form6', 'fuel_form7', 'fuel_form8', 'car_body0', 'car_body1', 'car_body2', 'car_body3',
                    'car_body4', 'car_body5', 'car_body6', 'car_body7', 'car_body8', 'car_body9', 'car_body10', 'car_body11', 
                    'series_level0', 'series_level1', 'series_level2', 'series_level3', 'series_level4', 'series_level5', 
                    'series_level6', 'series_level7', 'series_level8', 'series_level9', 'series_level10', 'series_level11', 
                    'series_level12', 'series_level13', 'series_level14', 'series_level15', 'series_level16', 'series_level17', 
                    'series_level18', 'series_level19', 'series_level20', 'series_level21', 'series_level22', 'series_level23', 
                    'series_level24', 'series_level25', 'series_level26', 'series_level27', 'series_level28', 'series_level29',
                    'series_level30', 'series_level31', 'series_level32', 'series_level33', 'series_level34', 'series_level35',
                    'series_level36', 'series_level37', 'series_level38', 'series_level39', 'series_level40', 'series_level41', 
                    'series_level42', 'series_level43', 'series_level44', 'series_level45', 'series_level46', 'series_level47', 
                    'series_level48', 'series_level49', 'series_level50', 'series_level51', 'series_level52', 'series_level53', 
                    'series_level54', 'series_level55', 'series_level56', 'series_level57', 'series_level58', 'series_level59', 
                    'series_level60', 'series_level61', 'wordvec0', 'wordvec1', 'wordvec2', 'wordvec3', 'wordvec4', 'wordvec5', 
                    'wordvec6', 'wordvec7', 'wordvec8', 'wordvec9', 'wordvec10', 'wordvec11', 'wordvec12', 'wordvec13', 'wordvec14',
                    'wordvec15', 'wordvec16', 'wordvec17', 'wordvec18', 'wordvec19', 'wordvec20', 'wordvec21', 'wordvec22', 'wordvec23',
                    'wordvec24', 'wordvec25', 'wordvec26', 'wordvec27', 'wordvec28', 'wordvec29', 'wordvec30', 'wordvec31', 'wordvec32', 
                    'wordvec33', 'wordvec34', 'wordvec35', 'wordvec36', 'wordvec37', 'wordvec38', 'wordvec39', 'wordvec40', 'wordvec41',
                    'wordvec42', 'wordvec43', 'wordvec44', 'wordvec45', 'wordvec46', 'wordvec47', 'wordvec48', 'wordvec49', 'wordvec50',
                    'wordvec51', 'wordvec52', 'wordvec53', 'wordvec54', 'wordvec55', 'wordvec56', 'wordvec57', 'wordvec58', 'wordvec59', 
                    'wordvec60', 'wordvec61', 'wordvec62', 'wordvec63', 'wordvec64', 'wordvec65', 'wordvec66', 'wordvec67', 'wordvec68', 
                    'wordvec69', 'wordvec70', 'wordvec71', 'wordvec72', 'wordvec73', 'wordvec74', 'wordvec75', 'wordvec76', 'wordvec77',
                    'wordvec78', 'wordvec79', 'wordvec80', 'wordvec81', 'wordvec82', 'wordvec83', 'wordvec84', 'wordvec85', 'wordvec86',
                    'wordvec87', 'wordvec88', 'wordvec89', 'wordvec90', 'wordvec91', 'wordvec92', 'wordvec93', 'wordvec94', 'wordvec95', 
                    'wordvec96', 'wordvec97', 'wordvec98', 'wordvec99']


label = 'residual'
fearture_columns_2 = ['rank_desc_A', 'rank_desc_B', 'rank_desc_C', 'rank_desc_D',
                       'rank_level_35', 'rank_level_50', 'rank_level_65', 'rank_level_80',
                       'rank_level_95', 'color_name_其他', 'color_name_多色', 'color_name_棕',
                       'color_name_灰', 'color_name_白', 'color_name_粉', 'color_name_紫',
                       'color_name_红', 'color_name_绿', 'color_name_蓝', 'color_name_银',
                       'color_name_黄', 'color_name_黑', 'tradetimes_-1', 'tradetimes_0',
                       'tradetimes_1', 'tradetimes_2', 'tradetimes_3', 'tradetimes_4',
                       'tradetimes_5', 'purpose_10', 'purpose_20', 'purpose_30', 'purpose_40',
                       'purpose_50']


In [68]:
logger.log("处理基础特征数据完成.............")

[2021-01-26 10:35:11,001] {<ipython-input-2-cbe05f263385>:85} INFO - 处理基础特征数据完成.............


# 常规方式训练模型

In [74]:
# data3m[['action_mode','model_code']].groupby('action_mode').count().reset_index()
data3m = data3m.loc[~data3m[label].isnull()]
# data3m = data3m.loc[data3m['order_status'] == 80]
data3m = data3m.loc[data3m[label]<0.99]
data3m['model_year'] = data3m['model_year'].map(lambda x:int(x))
data3m = data3m.loc[~data3m['province_name'].isnull()]

test_data = data3m.loc[(data3m['pay_time']>='2020-11-01') & 
                       (data3m['pay_time']<'2020-12-01') & 
                        (data3m['auction_mode'].isin(['2','4'])) & 
                       (data3m['price'] >0.1)
                      ]

train_data = data3m.loc[(data3m['pay_time']<'2020-11-01') & 
                        (data3m['pay_time']>'2016-01-01') & 
                        (data3m['price'] >0.1)]


In [75]:
print('车易拍数据量：',data3m.loc[data3m['is_version'] == 0].shape)
print(data3m.shape,train_data.shape,test_data.shape)

车易拍数据量： (171430, 390)
(428874, 390) (388754, 390) (4212, 390)


In [78]:
import lightgbm as lgb
gbm = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression',
                        learning_rate = 0.03, 
                        n_estimators = 5000,
                        max_depth = 12,
                        num_leaves = 100, 
                        subsample = 1, 
                        colsample_bytree = 0.34,
                        min_child_samples = 110, 
                        n_jobs = 8,
                        verbose = 1)

gbm.fit(train_data[fearture_columns+fearture_columns_2], train_data[label],
        eval_set = (test_data[fearture_columns+fearture_columns_2], test_data[label]),
        eval_metric = {'l1','l2'}, 
        early_stopping_rounds = 50, 
        verbose=True)



[1]	valid_0's l1: 0.126739	valid_0's l2: 0.0234805
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l1: 0.124331	valid_0's l2: 0.02259
[3]	valid_0's l1: 0.120538	valid_0's l2: 0.021301
[4]	valid_0's l1: 0.118433	valid_0's l2: 0.0206074
[5]	valid_0's l1: 0.114761	valid_0's l2: 0.0194176
[6]	valid_0's l1: 0.1116	valid_0's l2: 0.0183701
[7]	valid_0's l1: 0.108225	valid_0's l2: 0.0173312
[8]	valid_0's l1: 0.104882	valid_0's l2: 0.0163502
[9]	valid_0's l1: 0.101987	valid_0's l2: 0.0154943
[10]	valid_0's l1: 0.0989486	valid_0's l2: 0.0146546
[11]	valid_0's l1: 0.0962204	valid_0's l2: 0.0139403
[12]	valid_0's l1: 0.0940729	valid_0's l2: 0.0133177
[13]	valid_0's l1: 0.0915316	valid_0's l2: 0.0126363
[14]	valid_0's l1: 0.0887261	valid_0's l2: 0.0119338
[15]	valid_0's l1: 0.0860899	valid_0's l2: 0.011312
[16]	valid_0's l1: 0.0838944	valid_0's l2: 0.0107698
[17]	valid_0's l1: 0.0824423	valid_0's l2: 0.0104252
[18]	valid_0's l1: 0.0807646	valid_0's l2: 0.0100566
[19]	val

[154]	valid_0's l1: 0.0271568	valid_0's l2: 0.00126978
[155]	valid_0's l1: 0.0270553	valid_0's l2: 0.00126065
[156]	valid_0's l1: 0.0269752	valid_0's l2: 0.00125417
[157]	valid_0's l1: 0.0269768	valid_0's l2: 0.00125407
[158]	valid_0's l1: 0.0269484	valid_0's l2: 0.00125178
[159]	valid_0's l1: 0.0269021	valid_0's l2: 0.00124736
[160]	valid_0's l1: 0.0268735	valid_0's l2: 0.00124442
[161]	valid_0's l1: 0.0267812	valid_0's l2: 0.00123696
[162]	valid_0's l1: 0.0267356	valid_0's l2: 0.00123244
[163]	valid_0's l1: 0.0266197	valid_0's l2: 0.00122335
[164]	valid_0's l1: 0.0265986	valid_0's l2: 0.00122159
[165]	valid_0's l1: 0.0265083	valid_0's l2: 0.00121464
[166]	valid_0's l1: 0.0264662	valid_0's l2: 0.00121144
[167]	valid_0's l1: 0.0264674	valid_0's l2: 0.0012108
[168]	valid_0's l1: 0.0264578	valid_0's l2: 0.00120954
[169]	valid_0's l1: 0.0264289	valid_0's l2: 0.00120718
[170]	valid_0's l1: 0.0264504	valid_0's l2: 0.00120861
[171]	valid_0's l1: 0.0263982	valid_0's l2: 0.00120387
[172]	valid

[305]	valid_0's l1: 0.0218471	valid_0's l2: 0.000876179
[306]	valid_0's l1: 0.0218332	valid_0's l2: 0.000875246
[307]	valid_0's l1: 0.0218144	valid_0's l2: 0.000874089
[308]	valid_0's l1: 0.0218011	valid_0's l2: 0.000873161
[309]	valid_0's l1: 0.0217895	valid_0's l2: 0.000872334
[310]	valid_0's l1: 0.0218006	valid_0's l2: 0.00087283
[311]	valid_0's l1: 0.0217387	valid_0's l2: 0.000869375
[312]	valid_0's l1: 0.0217404	valid_0's l2: 0.000869378
[313]	valid_0's l1: 0.0217419	valid_0's l2: 0.00086927
[314]	valid_0's l1: 0.021729	valid_0's l2: 0.000868553
[315]	valid_0's l1: 0.0217298	valid_0's l2: 0.000868498
[316]	valid_0's l1: 0.0217339	valid_0's l2: 0.000868196
[317]	valid_0's l1: 0.0217156	valid_0's l2: 0.000867209
[318]	valid_0's l1: 0.0217045	valid_0's l2: 0.000866581
[319]	valid_0's l1: 0.0216376	valid_0's l2: 0.000863001
[320]	valid_0's l1: 0.0216195	valid_0's l2: 0.000861928
[321]	valid_0's l1: 0.0215522	valid_0's l2: 0.000858138
[322]	valid_0's l1: 0.0215002	valid_0's l2: 0.00085

[453]	valid_0's l1: 0.0201787	valid_0's l2: 0.000766537
[454]	valid_0's l1: 0.0201679	valid_0's l2: 0.000765784
[455]	valid_0's l1: 0.0201627	valid_0's l2: 0.000765385
[456]	valid_0's l1: 0.0201606	valid_0's l2: 0.000765014
[457]	valid_0's l1: 0.0201573	valid_0's l2: 0.000764751
[458]	valid_0's l1: 0.0201432	valid_0's l2: 0.000763924
[459]	valid_0's l1: 0.0201333	valid_0's l2: 0.000763425
[460]	valid_0's l1: 0.0201232	valid_0's l2: 0.00076261
[461]	valid_0's l1: 0.0201223	valid_0's l2: 0.0007624
[462]	valid_0's l1: 0.0201221	valid_0's l2: 0.000762412
[463]	valid_0's l1: 0.0201076	valid_0's l2: 0.000761382
[464]	valid_0's l1: 0.0200504	valid_0's l2: 0.000758624
[465]	valid_0's l1: 0.0200509	valid_0's l2: 0.000758554
[466]	valid_0's l1: 0.0200133	valid_0's l2: 0.000756741
[467]	valid_0's l1: 0.0200123	valid_0's l2: 0.000756458
[468]	valid_0's l1: 0.0199971	valid_0's l2: 0.00075547
[469]	valid_0's l1: 0.0199838	valid_0's l2: 0.00075487
[470]	valid_0's l1: 0.0199678	valid_0's l2: 0.0007538

[604]	valid_0's l1: 0.0190336	valid_0's l2: 0.000692693
[605]	valid_0's l1: 0.0190327	valid_0's l2: 0.000692385
[606]	valid_0's l1: 0.0190301	valid_0's l2: 0.000692152
[607]	valid_0's l1: 0.0190318	valid_0's l2: 0.000692097
[608]	valid_0's l1: 0.0190268	valid_0's l2: 0.000691748
[609]	valid_0's l1: 0.0190226	valid_0's l2: 0.000691494
[610]	valid_0's l1: 0.0190144	valid_0's l2: 0.000691029
[611]	valid_0's l1: 0.0190099	valid_0's l2: 0.000690729
[612]	valid_0's l1: 0.0190109	valid_0's l2: 0.00069063
[613]	valid_0's l1: 0.0190034	valid_0's l2: 0.00068996
[614]	valid_0's l1: 0.0189989	valid_0's l2: 0.000689638
[615]	valid_0's l1: 0.0189931	valid_0's l2: 0.000689109
[616]	valid_0's l1: 0.0189862	valid_0's l2: 0.000688748
[617]	valid_0's l1: 0.0189789	valid_0's l2: 0.000688244
[618]	valid_0's l1: 0.0189797	valid_0's l2: 0.000688131
[619]	valid_0's l1: 0.018972	valid_0's l2: 0.000687556
[620]	valid_0's l1: 0.0189661	valid_0's l2: 0.00068712
[621]	valid_0's l1: 0.018967	valid_0's l2: 0.0006871

...

[1197]	valid_0's l1: 0.0177269	valid_0's l2: 0.000601831
[1198]	valid_0's l1: 0.017728	valid_0's l2: 0.00060179
[1199]	valid_0's l1: 0.0177261	valid_0's l2: 0.000601713
[1200]	valid_0's l1: 0.0177247	valid_0's l2: 0.000601648
[1201]	valid_0's l1: 0.0177278	valid_0's l2: 0.000601793
[1202]	valid_0's l1: 0.0177245	valid_0's l2: 0.000601599
[1203]	valid_0's l1: 0.0177249	valid_0's l2: 0.000601609
[1204]	valid_0's l1: 0.017725	valid_0's l2: 0.000601562
[1205]	valid_0's l1: 0.0177242	valid_0's l2: 0.000601501
[1206]	valid_0's l1: 0.0177227	valid_0's l2: 0.000601493
[1207]	valid_0's l1: 0.0177222	valid_0's l2: 0.000601428
[1208]	valid_0's l1: 0.0177224	valid_0's l2: 0.000601377
[1209]	valid_0's l1: 0.0177216	valid_0's l2: 0.000601294
[1210]	valid_0's l1: 0.0177225	valid_0's l2: 0.000601229
[1211]	valid_0's l1: 0.0177225	valid_0's l2: 0.000601228
[1212]	valid_0's l1: 0.0177197	valid_0's l2: 0.000601053
[1213]	valid_0's l1: 0.0177194	valid_0's l2: 0.000601029
[1214]	valid_0's l1: 0.0177194	val

[1345]	valid_0's l1: 0.0176508	valid_0's l2: 0.000595567
[1346]	valid_0's l1: 0.0176516	valid_0's l2: 0.000595547
[1347]	valid_0's l1: 0.01765	valid_0's l2: 0.000595448
[1348]	valid_0's l1: 0.0176502	valid_0's l2: 0.000595468
[1349]	valid_0's l1: 0.0176508	valid_0's l2: 0.000595554
[1350]	valid_0's l1: 0.0176485	valid_0's l2: 0.000595383
[1351]	valid_0's l1: 0.0176429	valid_0's l2: 0.000595188
[1352]	valid_0's l1: 0.0176407	valid_0's l2: 0.000595015
[1353]	valid_0's l1: 0.0176436	valid_0's l2: 0.000595082
[1354]	valid_0's l1: 0.01764	valid_0's l2: 0.000594989
[1355]	valid_0's l1: 0.0176414	valid_0's l2: 0.000595013
[1356]	valid_0's l1: 0.0176413	valid_0's l2: 0.000595005
[1357]	valid_0's l1: 0.0176404	valid_0's l2: 0.000594999
[1358]	valid_0's l1: 0.0176393	valid_0's l2: 0.000594967
[1359]	valid_0's l1: 0.0176342	valid_0's l2: 0.000594686
[1360]	valid_0's l1: 0.017634	valid_0's l2: 0.000594679
[1361]	valid_0's l1: 0.0176302	valid_0's l2: 0.000594402
[1362]	valid_0's l1: 0.0176256	valid

[1496]	valid_0's l1: 0.0174871	valid_0's l2: 0.000585415
[1497]	valid_0's l1: 0.0174879	valid_0's l2: 0.000585451
[1498]	valid_0's l1: 0.0174938	valid_0's l2: 0.000585731
[1499]	valid_0's l1: 0.0174921	valid_0's l2: 0.000585659
[1500]	valid_0's l1: 0.017492	valid_0's l2: 0.000585681
[1501]	valid_0's l1: 0.0174935	valid_0's l2: 0.000585804
[1502]	valid_0's l1: 0.0174934	valid_0's l2: 0.000585798
[1503]	valid_0's l1: 0.0174931	valid_0's l2: 0.000585786
[1504]	valid_0's l1: 0.0174852	valid_0's l2: 0.000585366
[1505]	valid_0's l1: 0.0174859	valid_0's l2: 0.000585403
[1506]	valid_0's l1: 0.0174848	valid_0's l2: 0.000585293
[1507]	valid_0's l1: 0.0174841	valid_0's l2: 0.000585263
[1508]	valid_0's l1: 0.0174807	valid_0's l2: 0.000585031
[1509]	valid_0's l1: 0.0174774	valid_0's l2: 0.000584702
[1510]	valid_0's l1: 0.0174766	valid_0's l2: 0.000584603
[1511]	valid_0's l1: 0.0174791	valid_0's l2: 0.000584668
[1512]	valid_0's l1: 0.0174789	valid_0's l2: 0.00058466
[1513]	valid_0's l1: 0.0174783	va

[1643]	valid_0's l1: 0.017406	valid_0's l2: 0.000579826
[1644]	valid_0's l1: 0.0174145	valid_0's l2: 0.000580219
[1645]	valid_0's l1: 0.0174144	valid_0's l2: 0.000580271
[1646]	valid_0's l1: 0.0174083	valid_0's l2: 0.000580085
[1647]	valid_0's l1: 0.0174101	valid_0's l2: 0.000580119
[1648]	valid_0's l1: 0.017408	valid_0's l2: 0.00058003
[1649]	valid_0's l1: 0.0174137	valid_0's l2: 0.000580221
[1650]	valid_0's l1: 0.0174106	valid_0's l2: 0.000580079
[1651]	valid_0's l1: 0.0174091	valid_0's l2: 0.000580016
[1652]	valid_0's l1: 0.0174176	valid_0's l2: 0.000580381
[1653]	valid_0's l1: 0.0174278	valid_0's l2: 0.000580817
[1654]	valid_0's l1: 0.0174271	valid_0's l2: 0.000580751
[1655]	valid_0's l1: 0.0174261	valid_0's l2: 0.000580711
[1656]	valid_0's l1: 0.0174301	valid_0's l2: 0.000580916
[1657]	valid_0's l1: 0.01743	valid_0's l2: 0.000580913
[1658]	valid_0's l1: 0.0174321	valid_0's l2: 0.000581101
[1659]	valid_0's l1: 0.0174303	valid_0's l2: 0.000581027
[1660]	valid_0's l1: 0.0174283	valid

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.34,
       learning_rate=0.03, max_depth=12, min_child_samples=110,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=5000,
       n_jobs=8, num_leaves=100, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [79]:
feature_importances_df = pd.DataFrame(fearture_columns+fearture_columns_2,columns=['fearture'])
feature_importances_df['feature_importances'] = gbm.feature_importances_
feature_importances_df.sort_values(by='feature_importances',ascending=False)

,fearture,feature_importances
0,days,6262
41,license_month_log,4028
42,new_car_price_log,3518
38,keep_value,3401
1,license_year,2503
45,height,2116
44,length,1922
40,mile_log,1894
4,rate_count,1872
2,model_year,1860


In [80]:
from sklearn.metrics import r2_score
test_data['predict_0'] = gbm.predict(test_data[fearture_columns+fearture_columns_2].fillna(0))
train_data['predict_0'] = gbm.predict(train_data[fearture_columns+fearture_columns_2].fillna(0))


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [81]:
## 数据结果评估
test_data['差异'] = (test_data['predict_0']-test_data['residual'])/test_data['residual']
train_data['差异'] = (train_data['predict_0']-train_data['residual'])/train_data['residual']
print("测试估值。。。")
logger.log("精准估值模型 测试数据评估.............")
data3m_pinggu(test_data,'差异')
print("训练估值。。。")
logger.log("精准估值模型 训练数据评估.............")
data3m_pinggu(train_data,'差异')


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


测试估值。。。
[2021-01-26 10:46:34,054] {<ipython-input-2-cbe05f263385>:85} INFO - 精准估值模型 测试数据评估.............
P<3%:  0.3343
P<5%:  0.5109
P<8%:  0.7066
P<10%:  0.7809
P<20%:  0.9385
训练估值。。。
[2021-01-26 10:46:34,089] {<ipython-input-2-cbe05f263385>:85} INFO - 精准估值模型 训练数据评估.............


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


P<3%:  0.2926
P<5%:  0.4539
P<8%:  0.6308
P<10%:  0.7127
P<20%:  0.8944


(0.2926, 0.4539, 0.6308, 0.7127, 0.8944)

In [82]:
##======================================== ##
##       分别剔除国三和异常数据进行评估数据
##======================================== ##
print('数据量',test_data.shape)
print('剔除国三和异常数据后的数据量',test_data.loc[(test_data['environmental'] != '国III') 
                    & (test_data['delete_flag'] != 1) 
                    & (test_data['delete_flag_1'] != 1) 
                    & (test_data['delete_flag_2'] != 1)].shape)
print('剔除国三数据评估')
data3m_pinggu(test_data.loc[test_data['environmental'] != '国III'],'差异')

print('剔除国三和异常分别剔除数据评估')
data3m_pinggu(test_data.loc[(test_data['environmental'] != '国III') 
                            & (test_data['delete_flag'] != 1)],'差异')

data3m_pinggu(test_data.loc[(test_data['environmental'] != '国III') 
                            & (test_data['delete_flag'] != 1) 
                            & (test_data['delete_flag_1'] != 1)],'差异')

data3m_pinggu(test_data.loc[(test_data['environmental'] != '国III') 
                            & (test_data['delete_flag'] != 1) 
                            & (test_data['delete_flag_1'] != 1) 
                            & (test_data['delete_flag_2'] != 1)],'差异')

data3m_pinggu(test_data.loc[(test_data['delete_flag'] != 1) 
                            & (test_data['delete_flag_1'] != 1) 
                            & (test_data['delete_flag_2'] != 1)],'差异')



数据量 (4212, 392)
剔除国三和异常数据后的数据量 (3585, 392)
剔除国三数据评估
P<3%:  0.3425
P<5%:  0.5223
P<8%:  0.7177
P<10%:  0.7938
P<20%:  0.9455
剔除国三和异常分别剔除数据评估
P<3%:  0.3425
P<5%:  0.5223
P<8%:  0.7177
P<10%:  0.7938
P<20%:  0.9455
P<3%:  0.3411
P<5%:  0.5195
P<8%:  0.7165
P<10%:  0.7946
P<20%:  0.9474
P<3%:  0.3431
P<5%:  0.5213
P<8%:  0.7177
P<10%:  0.795
P<20%:  0.9473
P<3%:  0.3363
P<5%:  0.5116
P<8%:  0.7073
P<10%:  0.7832
P<20%:  0.9407


(0.3363, 0.5116, 0.7073, 0.7832, 0.9407)

In [83]:
#### 导出模型数据
for i in range(15):
    data3m = data3m.rename(columns={"country_id"+str(i):"Country_Id"+str(i)})
fearture_columns = ['days', 'license_year', 'model_year', 'rate', 'rate_count', 'per_mile', 'year_err',
                    'province_name0', 'province_name1', 'province_name2', 'province_name3', 'province_name4',
                    'province_name5', 'province_name6', 'province_name7', 'province_name8', 'province_name9', 
                    'province_name10', 'province_name11', 'province_name12', 'province_name13', 'province_name14',
                    'province_name15', 'province_name16', 'province_name17', 'province_name18', 'province_name19', 
                    'province_name20', 'province_name21', 'province_name22', 'province_name23', 'province_name24', 
                    'province_name25', 'province_name26', 'province_name27', 'province_name28', 'province_name29', 
                    'province_name30', 'keep_value', 'quality', 'mile_log', 'license_month_log', 'new_car_price_log',
                    'wheel_base', 'length', 'height', 'width', 'max_torque', 'max_power', 'engine_volume_l', 'cylinder_number', 
                    'seat_number_top', 'driving_mode0', 'driving_mode1', 'driving_mode2', 'driving_mode3', 'driving_mode4', 
                    'driving_mode5', 'driving_mode6', 'driving_mode7', 'driving_mode8', 'driving_mode9', 'gear_box_type0', 
                    'gear_box_type1', 'gear_box_type2', 'gear_box_type3', 'gear_box_type4', 'gear_box_type5', 'gear_box_type6', 
                    'gear_box_type7', 'gear_box_type8', 'gear_box_type9', 'gear_box_type10', 'Country_Id0', 'Country_Id1', 
                    'Country_Id2', 'Country_Id3', 'Country_Id4', 'Country_Id5', 'Country_Id6', 'Country_Id7', 'Country_Id8', 
                    'Country_Id9', 'Country_Id10', 'Country_Id11', 'Country_Id12', 'Country_Id13', 'Country_Id14', 'import_type0', 
                    'import_type1', 'intake_type0', 'intake_type1', 'intake_type2', 'intake_type3', 'intake_type4', 'intake_type5', 
                    'intake_type6', 'intake_type7', 'fuel_form0', 'fuel_form1', 'fuel_form2', 'fuel_form3', 'fuel_form4', 
                    'fuel_form5', 'fuel_form6', 'fuel_form7', 'fuel_form8', 'car_body0', 'car_body1', 'car_body2', 'car_body3',
                    'car_body4', 'car_body5', 'car_body6', 'car_body7', 'car_body8', 'car_body9', 'car_body10', 'car_body11', 
                    'series_level0', 'series_level1', 'series_level2', 'series_level3', 'series_level4', 'series_level5', 
                    'series_level6', 'series_level7', 'series_level8', 'series_level9', 'series_level10', 'series_level11', 
                    'series_level12', 'series_level13', 'series_level14', 'series_level15', 'series_level16', 'series_level17', 
                    'series_level18', 'series_level19', 'series_level20', 'series_level21', 'series_level22', 'series_level23', 
                    'series_level24', 'series_level25', 'series_level26', 'series_level27', 'series_level28', 'series_level29',
                    'series_level30', 'series_level31', 'series_level32', 'series_level33', 'series_level34', 'series_level35',
                    'series_level36', 'series_level37', 'series_level38', 'series_level39', 'series_level40', 'series_level41', 
                    'series_level42', 'series_level43', 'series_level44', 'series_level45', 'series_level46', 'series_level47', 
                    'series_level48', 'series_level49', 'series_level50', 'series_level51', 'series_level52', 'series_level53', 
                    'series_level54', 'series_level55', 'series_level56', 'series_level57', 'series_level58', 'series_level59', 
                    'series_level60', 'series_level61', 'wordvec0', 'wordvec1', 'wordvec2', 'wordvec3', 'wordvec4', 'wordvec5', 
                    'wordvec6', 'wordvec7', 'wordvec8', 'wordvec9', 'wordvec10', 'wordvec11', 'wordvec12', 'wordvec13', 'wordvec14',
                    'wordvec15', 'wordvec16', 'wordvec17', 'wordvec18', 'wordvec19', 'wordvec20', 'wordvec21', 'wordvec22', 'wordvec23',
                    'wordvec24', 'wordvec25', 'wordvec26', 'wordvec27', 'wordvec28', 'wordvec29', 'wordvec30', 'wordvec31', 'wordvec32', 
                    'wordvec33', 'wordvec34', 'wordvec35', 'wordvec36', 'wordvec37', 'wordvec38', 'wordvec39', 'wordvec40', 'wordvec41',
                    'wordvec42', 'wordvec43', 'wordvec44', 'wordvec45', 'wordvec46', 'wordvec47', 'wordvec48', 'wordvec49', 'wordvec50',
                    'wordvec51', 'wordvec52', 'wordvec53', 'wordvec54', 'wordvec55', 'wordvec56', 'wordvec57', 'wordvec58', 'wordvec59', 
                    'wordvec60', 'wordvec61', 'wordvec62', 'wordvec63', 'wordvec64', 'wordvec65', 'wordvec66', 'wordvec67', 'wordvec68', 
                    'wordvec69', 'wordvec70', 'wordvec71', 'wordvec72', 'wordvec73', 'wordvec74', 'wordvec75', 'wordvec76', 'wordvec77',
                    'wordvec78', 'wordvec79', 'wordvec80', 'wordvec81', 'wordvec82', 'wordvec83', 'wordvec84', 'wordvec85', 'wordvec86',
                    'wordvec87', 'wordvec88', 'wordvec89', 'wordvec90', 'wordvec91', 'wordvec92', 'wordvec93', 'wordvec94', 'wordvec95', 
                    'wordvec96', 'wordvec97', 'wordvec98', 'wordvec99']

label = 'residual'

from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml
import lightgbm as lgb

pipeline = PMMLPipeline([("regressor", lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression',
                        learning_rate = 0.03, 
                        n_estimators = 5000,
                        max_depth = 12,
                        num_leaves = 100, 
                        subsample = 1, 
                        colsample_bytree = 0.34,
                        min_child_samples = 110, 
                        n_jobs = 8,
                        verbose = 1))])

pipeline.fit(data3m[fearture_columns+fearture_columns_2], data3m[label])

print("保存模型。。。。。")
joblib.dump(pipeline,curr_dir+'enterprise2_cyp_auction_python')

ph = curr_dir+"enterprise2_cyp_auction.pmml"
sklearn2pmml(pipeline, ph, with_repr = True)

## 到处测试数据
ph = curr_dir+"enterprise2_cyp_auction.pmml"
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation_cyp/模型更新_n/"+curr_date+"/enterprise2_cyp_auction.pmml", ph)
print("保存模型文件ok........")

## 到处测试数据
ph = curr_dir+"enterprise2_cyp_auction_python"
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation_cyp/模型更新_n/"+curr_date+"/enterprise2_cyp_auction_python", ph)
print("保存模型文件ok........")

# car = pd.read_csv(curr_dir+"2020-11-02版车型参数及独热编码.csv", header = 0, low_memory=False)
sql='''
select * from db_data.enterprise2_car_parameter_onehot 
where ds = date_sub("'''+car_model_date+'''",1)
'''
dtype={'city_code':str}
model = read_from_hive2('mid_car_cyp_paimai_order_dd',sql,dtype)

model_columns = []
for col in model.columns:
    if len(col.split('.')) > 1:
        model_columns.append(col.split('.')[1])

    else:
        model_columns.append(col)
model.columns = model_columns 

car = model.drop(['driving_mode', 'gear_box_type', 'country_id',
                'import_type', 'intake_type', 'fuel_form',
                'year_1', 'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7', 'year_8',
                'year_9', 'year_10', 'year_11', 'year_12', 'year_13', 'year_14', 'year_15',
                'year_16', 'car_body', 'rate', 'rate_count', 'series_level', 'level0',
                'level1', 'level2','level3','level4','level5','level6','level7','level8'], axis=1)
ph = curr_dir+"enterprise2_car_parameter_onehot.csv"
car.to_csv(ph)
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation_cyp/模型更新_n/"+curr_date+"/enterprise2_car_parameter_onehot.csv", ph)
print("enterprise2_car_parameter_onehot 文件ok........")

# vec = pd.read_csv(curr_dir+"2020-11-02版-TF-IDF权重的词向量.csv", header = 0, low_memory=False)

sql='''
select * from db_data.enterprise2_model_name_word2vec 
where ds = date_sub("'''+car_model_date+'''",1)
'''
dtype={'city_code':str}
vec = read_from_hive2('vec',sql,dtype)

vec_columns = []
for col in vec.columns:
    if len(col.split('.')) > 1:
        vec_columns.append(col.split('.')[1])

    else:
        vec_columns.append(col)
vec.columns = vec_columns 
vec.info()
ph = curr_dir+"enterprise2_model_name_word2vec.csv"
vec.to_csv(ph)
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation_cyp/模型更新_n/"+curr_date+"/enterprise2_model_name_word2vec.csv", ph)
print("enterprise2_model_name_word2vec 文件ok........")

car_data_columns= ["country_id","manufacturer","battery_quality_mile","battery_quality_time","brand_code","brand_name","car_body",
"cylinder_number","driving_mode","engine_volume_l","fuel_form","gear_box_type","guide_price","height","import_type",
"intake_type","length","level","max_power","max_torque","model_code","model_name","quality_mile","quality_time","rate",
"rate_count","seat_number_top","series_code","series_level","series_name","wheel_base","width","year","down_market_time",
"production_time","shutdown_time","time_to_market","year_1","year_2","year_3","year_4","year_5","year_6","year_7","year_8",
"year_9","year_10","year_11","year_12","year_13","year_14","year_15","year_16"]
q_res = pd.read_csv(curr_dir+'all_car_model_sep_word_to_sql',header=None,sep='\t',names=car_data_columns)
q_res = q_res[['model_code','series_code','brand_code']]
# keep_value = pd.read_csv(curr_dir+"2020-11-02版车型参数及独热编码.csv", header = 0, low_memory=False)
keep_value = model[['model_code', 'year_1', 'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7', 'year_8',
                         'year_9', 'year_10', 'year_11', 'year_12', 'year_13', 'year_14', 'year_15',
                         'year_16', 'rate', 'rate_count']]

keep_value=pd.merge(keep_value,q_res,on='model_code',how='left')
ph = curr_dir+"enterprise2_keep_value_and_rate.csv"
keep_value.to_csv(ph)
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation_cyp/模型更新_n/"+curr_date+"/enterprise2_keep_value_and_rate.csv", ph)
print("enterprise2_keep_value_and_rate 文件ok........")


保存模型。。。。。
[2021-01-26 10:52:00,329] {oss_hook:28} INFO - Done. Loaded the key algorithm/qiongjiu/valuation_cyp/模型更新_n/2021-01-25/enterprise2_cyp_auction.pmml .
保存模型文件ok........


In [95]:
data3m_cyp = data3m.loc[data3m['is_version'] ==0]


(171230, 387)

In [98]:
## 计算置信度数据并导出
data3m_n = data3m.loc[data3m['is_version'] ==0]
data3m_n['predict_1'] = pipeline.predict(data3m_n[fearture_columns+fearture_columns_2].fillna(0))
data3m_n['差异'] = (data3m_n['predict_1']-data3m_n['residual'])/data3m_n['residual']

test_data_03_count = data3m_n.loc[(data3m_n['差异'] <= 0.03) & (data3m_n['差异'] >= -0.03)][['model_code','差异']].\
 groupby("model_code").count().reset_index().rename(columns={"差异":"count_03"})

test_data_05_count = data3m_n.loc[(data3m_n['差异'] <= 0.05) & (data3m_n['差异'] >= -0.05)][['model_code','差异']].\
 groupby("model_code").count().reset_index().rename(columns={"差异":"count_05"})

test_data_08_count = data3m_n.loc[(data3m_n['差异'] <= 0.08) & (data3m_n['差异'] >= -0.08)][['model_code','差异']].\
 groupby("model_code").count().reset_index().rename(columns={"差异":"count_08"})

test_data_10_count = data3m_n.loc[(data3m_n['差异'] <= 0.10) & (data3m_n['差异'] >= -0.10)][['model_code','差异']].\
 groupby("model_code").count().reset_index().rename(columns={"差异":"count_10"})

test_data_count = data3m_n[['model_code','差异']].\
groupby(["model_code"]).count().reset_index().rename(columns={"差异":"count"})

test_data_count = pd.merge(test_data_count,test_data_03_count,on='model_code',how='left')
test_data_count = pd.merge(test_data_count,test_data_05_count,on='model_code',how='left')
test_data_count = pd.merge(test_data_count,test_data_08_count,on='model_code',how='left')
test_data_count = pd.merge(test_data_count,test_data_10_count,on='model_code',how='left')

test_data_count['count_3%_rate'] = test_data_count['count_03'].fillna(0) / test_data_count['count']
test_data_count['count_5%_rate'] = test_data_count['count_05'].fillna(0) / test_data_count['count']
test_data_count['count_8%_rate'] = test_data_count['count_08'].fillna(0) / test_data_count['count']
test_data_count['count_10%_rate'] = test_data_count['count_10'].fillna(0) / test_data_count['count']

file_name = "车易拍精准估值分段置信度.xlsx"
test_data_count.to_excel(curr_dir+file_name)
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
# oss.put_file("algorithm/qiongjiu/valuation_cyp/模型更新/2020-11-02/"+file_name, curr_dir+file_name)
oss.put_file("algorithm/qiongjiu/valuation_cyp/模型更新_n/"+curr_date+'/'+file_name, curr_dir+file_name)



/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


[2021-01-25 17:32:51,492] {oss_hook:28} INFO - Done. Loaded the key algorithm/qiongjiu/valuation_cyp/模型更新_n/2021-01-25/车易拍精准估值分段置信度.xlsx .
